<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Downloading-the-dataset" data-toc-modified-id="Downloading-the-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Downloading the dataset</a></span><ul class="toc-item"><li><span><a href="#Audio-dataset" data-toc-modified-id="Audio-dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Audio dataset</a></span><ul class="toc-item"><li><span><a href="#loadign-the-catalog-of-the-cleaned-data-for-train-dataset" data-toc-modified-id="loadign-the-catalog-of-the-cleaned-data-for-train-dataset-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>loadign the catalog of the cleaned data for train dataset</a></span></li><li><span><a href="#some-packages-to-download-the-audio-files" data-toc-modified-id="some-packages-to-download-the-audio-files-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>some packages to download the audio files</a></span></li><li><span><a href="#normalisation-of-the-audio-tracks" data-toc-modified-id="normalisation-of-the-audio-tracks-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>normalisation of the audio tracks</a></span></li></ul></li><li><span><a href="#Video-dataset" data-toc-modified-id="Video-dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Video dataset</a></span><ul class="toc-item"><li><span><a href="#face-detection-from-the-raw-frame" data-toc-modified-id="face-detection-from-the-raw-frame-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>face detection from the raw frame</a></span></li></ul></li></ul></li></ul></div>

# Downloading the dataset

## Audio dataset

In [6]:
# download and preprocess the data from AVspeech dataset
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
# pip install librosa
sys.path.append("../cocktail_effect/data")  # path to the AVHandler.py
import AVHandler as avh

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def m_link(youtube_id):
    # return the youtube actual link
    link = 'https://www.youtube.com/watch?v='+ youtube_id
    return link

def m_audio(loc, name, cat, start_idx, end_idx):
    # make concatenated audio following by the catalog from AVSpeech
    # loc       | the location for file to store
    # name      | name for the wav mix file
    # cat       | the catalog with audio link and time
    # start_idx | the starting index of the audio to download and concatenate
    # end_idx   | the ending index of the audio to download and concatenate

    for i in range(start_idx, end_idx):
        f_name = name+str(i)
        link = m_link(cat.loc[i,'link'])
        start_time = cat.loc[i,'start_time']
        end_time = start_time + 3.0
        avh.download(loc,f_name,link)
        avh.cut(loc,f_name,start_time,end_time)

### loadign the catalog of the cleaned data for train dataset

In [3]:
header = ["link", "start_time", "end_time", "x_coord", "y_coord"]
cat_train = pd.read_csv('../raw_data/avspeech_train.csv', names=header)
#cat_test = pd.read_csv('catalog/avspeech_test.csv', names=header)
cat_train.head()

,link,start_time,end_time,x_coord,y_coord
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


### some packages to download the audio files

SoX is a cross-platform (Windows, Linux, MacOS X, etc.) command line utility that can convert various formats of computer audio files in to other formats. more on [SoX](http://sox.sourceforge.net/)

In [4]:
# pip install youtube-dl
# !brew install ffmpeg   # for converting, recording and streaming audio and videos
# !brew install sox      # SoX - Sound eXchange

In [8]:
# avh.mkdir('audio_train')  # audio_train model is created in the notebook folder
m_audio('audio_train','audio_train',cat_train, 2, 3)

sh: line 0: cd: audio_train: No such file or directory


[youtube] Y8HMIm8mdns: Downloading webpage
[youtube] Y8HMIm8mdns: Downloading MPD manifest
[download] Destination: oaudio_train2.wav
[download] 100% of 6.42MiB in 01:5728KiB/s ETA 00:00


ERROR: ffprobe/avprobe and ffmpeg/avconv not found. Please install one.
sh: ffmpeg: command not found
sh: line 0: cd: audio_train: No such file or directory
sh: sox: command not found
rm: audio_train2.wav: No such file or directory


### normalisation of the audio tracks

In the AVHandler sampling rate of the audio files are set to 16000. 

In [ ]:
import audio_norm
audio_norm  # This will make a new folder (norm_audio_trian) with normalised audio tracks

## Video dataset

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
sys.path.append("../models/lib")


In [ ]:
def video_download(loc,cat,start_idx,end_idx):
    # Only download the video from the link
    # loc        | the location for downloaded file
    # v_name     | the name for the video file
    # cat        | the catalog with audio link and time
    # start_idx  | the starting index of the video to download
    # end_idx    | the ending index of the video to download

    for i in range(start_idx,end_idx):
        command = 'cd %s;' % loc
        f_name = str(i)
        link = avh.m_link(cat.loc[i, 'link'])
        start_time = cat.loc[i, 'start_time']
        end_time = start_time + 3.0
        start_time = datetime.timedelta(seconds=start_time)
        end_time = datetime.timedelta(seconds=end_time)
        command += 'ffmpeg -i $(youtube-dl -f ”mp4“ --get-url ' + link + ') ' + '-c:v h264 -c:a copy -ss %s -to %s %s.mp4' \
                % (start_time, end_time, f_name)
        os.system(command)
    
def generate_frames(loc, start_idx, end_idx):
    # get frames for each video clip
    # loc        | the location of video clip
    # v_name     | v_name = 'clip_video_train'
    # start_idx  | the starting index of the training sample
    # end_idx    | the ending index of the training sample

    avh.mkdir('frames')
    for i in range(start_idx, end_idx):
        command = 'cd %s;' % loc
        f_name = str(i)
        command += 'ffmpeg -i %s.mp4 -y -f image2  -vframes 75 ../frames/%s-%%02d.jpg' % (
            f_name, f_name)
        os.system(command)


def download_video_frames(loc, cat, start_idx, end_idx, rm_video):
    # Download each video and convert to frames immediately, can choose to remove video file
    # loc        | the location for downloaded file
    # cat        | the catalog with audio link and time
    # start_idx  | the starting index of the video to download
    # end_idx    | the ending index of the video to download
    # rm_video   | boolean value for delete video and only keep the frames

    avh.mkdir('frames')
    for i in range(start_idx, end_idx + 1):
        command = 'cd %s;' % loc
        f_name = str(i)
        link = avh.m_link(cat.loc[i, 'link'])
        start_time = cat.loc[i, 'start_time']
        end_time = start_time + 3.0
        start_time = datetime.timedelta(seconds=start_time)
        end_time = datetime.timedelta(seconds=end_time)
        command += 'ffmpeg -i $(youtube-dl -f ”mp4“ --get-url ' + link + ') ' + '-c:v h264 -c:a copy -ss %s -to %s %s.mp4;' \
                   % (start_time, end_time, f_name)
        command += 'ffmpeg -i %s.mp4 -vf fps=25 ../frames/%s-%%02d.jpg;' % (
            f_name, f_name)
        if rm_video:
            command += 'rm %s.mp4' % f_name
        os.system(command)

In [ ]:
avh.mkdir('video_train')

# download video , convert to images separately
#avh.video_download(loc='video_train',v_name='video_train',cat=cat_train,start_idx=2,end_idx=4)
#avh.generate_frames(loc='video_train',v_name='clip_video_train',start_idx=2,end_idx=4)

# download each video and convert to frames immediately
download_video_frames(loc='video_train',
                      cat=cat_train,
                      start_idx=1,
                      end_idx=2,
                      rm_video=True)

### face detection from the raw frame

In [ ]:
from mtcnn import MTCNN

In [ ]:
header = ["link", "start_time", "end_time", "x_coord", "y_coord"]
cat_train = pd.read_csv('../raw_data/avspeech_train.csv', names=header)
#cat_test = pd.read_csv('catalog/avspeech_test.csv', names=header)
cat_train.head()

import cv2
import pandas as pd
import os
cat_train = pd.read_csv('../raw_data/avspeech_train.csv')
frame_path = './frames/'
output_dir = './face_input'
detect_range = (1, 2)